# Data Transormation

## Install/Import packages & define key varribles and functions

In [1]:
# Run install script
# %chmod +x setup_jupyterlab.sh
# %./setup_jupyterlab.sh

# Import necessary libraries for the script to function.
import pandas as pd
import csv, json, re, os, venn, sys, math, base64, io
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Patch
import matplotlib.patches as mpatches

from datetime import datetime

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import warnings

from sklearn.decomposition import PCA

from functools import partial
import seaborn as sns
from scipy.stats import pearsonr
from itertools import combinations

from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import plotly.graph_objects as go

In [2]:
merged_df = pd.read_csv('Merged_Dataframe_20241229_174216.csv')
merged_df

,unique ID,Sequence,Master Protein Accessions,Positions in Proteins,start,stop,Peptide Groups Peptide Group ID,Checked,Confidence,Annotated Sequence,...,SEM_I_Gastric,SEM_I_Intestinal,SEM_J_Feed,SEM_J_Gastric,SEM_J_Intestinal,SEM_Bovine_Feed,SEM_Bovine_Gastric,SEM_Bovine_Intestinal,SEM_Bovine,SEM_A1
0,RELEELNVPGE,RELEELNVPGE,P02666,P02666 [16-26],16.0,26.0,14178,False,High,[A].RELEELNVPGE.[I],...,9.759447e+06,NaN,8.640665e+06,5.479655e+06,NaN,1.238679e+07,9.621040e+06,8.279631e+06,7.325243e+06,2.220128e+06
1,RELEELNVPG,RELEELNVPG,P02666,P02666 [16-25],16.0,25.0,3495,False,High,[A].RELEELNVPG.[E],...,4.356191e+06,NaN,1.927371e+06,3.096565e+06,NaN,4.587615e+06,3.290370e+06,7.682908e+06,2.793666e+06,1.870235e+06
2,SPPEINTVQVTSTAV,SPPEINTVQVTSTAV,P02668,P02668 [176-190],176.0,190.0,2752,False,High,[E].SPPEINTVQVTSTAV.[-],...,5.902470e+06,NaN,5.450790e+06,7.588260e+05,NaN,2.679356e+06,2.745346e+06,1.246571e+07,1.915543e+06,3.468302e+06
3,ILNKPEDETHLEAQPTDASAQF,ILNKPEDETHLEAQPTDASAQF,P80195,P80195 [19-40],19.0,40.0,3336,False,High,[A].ILNKPEDETHLEAQPTDASAQF.[I],...,4.770135e+07,NaN,1.295161e+06,3.394561e+07,96336.036632,2.844237e+06,4.030563e+07,2.212346e+07,2.113881e+07,7.091215e+07
4,SPEVIESPPEINTVQVTSTAV_1xPhospho [S1],SPEVIESPPEINTVQVTSTAV,P02668,P02668 [170-190],170.0,190.0,5386,False,High,[D].SPEVIESPPEINTVQVTSTAV.[-],...,9.757782e+04,NaN,1.793615e+06,3.225499e+05,NaN,1.702542e+06,1.165906e+06,NaN,1.033437e+06,1.086924e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4055,AVPYPQRDMPIQAFL_1xOxidation [M9],AVPYPQRDMPIQAFL,P02666,P02666 [192-206],192.0,206.0,43362,False,High,[K].AVPYPQRDMPIQAFL.[L],...,1.224428e+05,NaN,1.060382e+04,2.206344e+04,NaN,1.056864e+05,5.418654e+04,NaN,5.564213e+04,NaN
4056,LAMAASDISLLDAQSAPLRV,LAMAASDISLLDAQSAPLRV,P02754; Q9BDG3,P02754 [38-57]; Q9BDG3 [7-26],NaN,NaN,43367,False,High,[S].LAMAASDISLLDAQSAPLRV.[Y],...,NaN,NaN,NaN,2.426932e+03,NaN,NaN,2.545395e+04,3.314839e+04,2.805262e+04,5.509304e+04
4057,YQGPIVLNPWDQVKRNAVPITPTL,YQGPIVLNPWDQVKRNAVPITPTL,P02663,P02663 [115-138],115.0,138.0,43440,False,High,[L].YQGPIVLNPWDQVKRNAVPITPTL.[N],...,4.440056e+04,NaN,NaN,2.638909e+04,NaN,1.721444e+04,9.440658e+04,NaN,8.686221e+04,2.812042e+05
4058,NQFLPYPYYAKPAAVRSPAQILQ,NQFLPYPYYAKPAAVRSPAQILQ,P02668,P02668 [74-96],74.0,96.0,42011,False,High,[N].NQFLPYPYYAKPAAVRSPAQILQ.[W],...,NaN,NaN,NaN,6.063436e+04,NaN,NaN,1.762571e+04,NaN,1.762571e+04,3.992836e+04


In [3]:
# Automatically load group_data.json from the specified directory
file_path = os.path.join('group_data3.json')
if os.path.exists(file_path):
    with open(file_path, 'r') as json_file:
        group_data = json.load(json_file)
    print(f"Loaded group data from {file_path}")
    print(f"{len(group_data)} groups imported from file")

else:
    print(f"File {file_path} does not exist. Please check the path.")
    print(f"Importing group data from defined in earlier in the current session ")
    from my_functions_datatransformation import group_data
global group_data
group_data = {key: group_data[key] for key in list(group_data.keys())[3:9] + list(group_data.keys())[12:42]}

# Loop through the group_data dictionary and apply the renaming

group_data

Loaded group data from group_data3.json
47 groups imported from file


{'4': {'grouping_variable': 'Human_Milk_Intestinal',
  'abundance_columns': ['HM1_I', 'HM2_I', 'HM3_I', 'HM4_I']},
 '5': {'grouping_variable': 'Human_Milk_Gastric',
  'abundance_columns': ['HM1_G', 'HM2_G', 'HM3_G', 'HM4_G']},
 '6': {'grouping_variable': 'Human_Milk_Feed',
  'abundance_columns': ['HM1_F', 'HM2_F', 'HM3_F', 'HM4_F']},
 '7': {'grouping_variable': 'Control_Feed',
  'abundance_columns': ['A1_F', 'A2_F', 'A3_F']},
 '8': {'grouping_variable': 'Control_Gastric',
  'abundance_columns': ['A1_G', 'A2_G', 'A3_G']},
 '9': {'grouping_variable': 'Control_Intestinal',
  'abundance_columns': ['A1_I', 'A2_I', 'A3_I']},
 '13': {'grouping_variable': 'A1_Feed',
  'abundance_columns': ['A1_1_F', 'A1_2_F', 'A1_3_F']},
 '14': {'grouping_variable': 'A1_Gastric',
  'abundance_columns': ['A1_1_G', 'A1_2_G', 'A1_3_G']},
 '15': {'grouping_variable': 'A1_Intestinal',
  'abundance_columns': ['A1_1_I', 'A1_2_I', 'A1_3_I']},
 '16': {'grouping_variable': 'B_Feed',
  'abundance_columns': ['B1_F', 'B2_F

# Data Visulization

## Correlation

In [4]:

def calculate_all_correlations(df, group_data):
    correlation_results = []

    # Extract the relevant 'Average_Abundance' columns for each group
    Avg_columns = {
        group_info['grouping_variable']: f"Avg_{group_info['grouping_variable']}"
        for group_info in group_data.values()
        if f"Avg_{group_info['grouping_variable']}" in merged_df.columns
    }

   
    # Generate all unique pairs of groupings for comparison
    for (group1, col1), (group2, col2) in combinations(Avg_columns.items(), 2):
           #print(group1, group2,len(df))
        # Calculate Pearson correlation between the two groups
        # Apply log10 transformation
        log_col1 = np.log10(df[col1])
        log_col2 = np.log10(df[col2])
        correlation = log_col1.corr(log_col2)
        # Round the correlation to the thousandth place
        correlation = round(correlation, 3)
        correlation_results.append((group1, group2, correlation))

    # Convert the results into a DataFrame
    correlation_df = pd.DataFrame(correlation_results, columns=['Base ID Group 1', 'Base ID Group 2', 'Correlation'])

    # Save the DataFrame to a CSV file
    correlation_df.to_csv('all_correlations.csv', index=False)

    return correlation_df

# Example usage
correlation_df = calculate_all_correlations(merged_df, group_data)


In [5]:
def plot_interactive_correlation(df, group_data):
    """
    Creates an interactive correlation plot with simplified function filtering.
    """
    # Create dropdown widgets for group selection using group names
    group_names = [group_info['grouping_variable'] for group_info in group_data.values()]
    
    # Extract unique function components and count their occurrences
    all_functions = df['function'].dropna().unique()
    function_counts = {}
    
    # Count occurrences of each function component
    for func in df['function'].dropna():
        components = [f.strip() for f in func.replace(';', ',').split(',')]
        for component in components:
            function_counts[component] = function_counts.get(component, 0) + 1
    
    # Filter for functions with more than 1 occurrence
    valid_functions = [func for func, count in function_counts.items() if count > 1]
    
    # Sort function options and add 'All' and 'Unknown'
    function_options = ['All', 'Unknown'] + sorted(valid_functions)
    
    # Create widgets
    group1_widget = widgets.Dropdown(
        options=group_names,
        description='Group 1:',
        value=group_names[0],
        layout=widgets.Layout(width='400px')
    )
    
    group2_widget = widgets.Dropdown(
        options=group_names,
        description='Group 2:',
        value=group_names[1],
        layout=widgets.Layout(width='400px')
    )
    
    function_widget = widgets.Dropdown(
        options=function_options,
        description='Function:',
        value='All',
        layout=widgets.Layout(width='400px')
    )
    
    color_widget = widgets.ColorPicker(
        concise=False,
        description='Plot Color:',
        value='#0072C6',
        layout=widgets.Layout(width='400px')
    )
    
    xlabel_widget = widgets.Text(
        description='X Label:',
        placeholder='Enter x-axis label',
        layout=widgets.Layout(width='400px')
    )
    
    ylabel_widget = widgets.Text(
        description='Y Label:',
        placeholder='Enter y-axis label',
        layout=widgets.Layout(width='400px')
    )
    
    # Output widgets
    info_output = widgets.Output()
    plot_output = widgets.Output()
    
    def create_plot(change=None):
        group1 = group1_widget.value
        group2 = group2_widget.value
        function_filter = function_widget.value
        color = color_widget.value
        xlabel = xlabel_widget.value
        ylabel = ylabel_widget.value
        
        with plot_output:
            plot_output.clear_output(wait=True)
            
            if group1 == group2:
                display(HTML(f'<b style="color:red;">Error: Please select different groups for comparison.</b>'))
                return
                
            col1 = f"Avg_{group1}"
            col2 = f"Avg_{group2}"
            
            if col1 not in df.columns or col2 not in df.columns:
                display(HTML(f'<b style="color:red;">Error: Skipping plot for {group1} vs {group2} as one or both columns are missing.</b>'))
                return
            
            # Filter data based on selected function
            working_df = df.copy()
            if function_filter == 'Unknown':
                working_df = working_df[working_df['function'].isna()]
            elif function_filter != 'All':
                # Filter for rows where the function contains the selected component
                working_df = working_df[
                    working_df['function'].fillna('').str.contains(
                        function_filter, 
                        case=False, 
                        na=False
                    )
                ]
            
            # Filter for positive values
            filtered_df = working_df[(working_df[col1] > 0) & (working_df[col2] > 0)].copy()
            
            if len(filtered_df) == 0:
                display(HTML(f'<b style="color:red;">No data points available for the selected combination.</b>'))
                return
            
            # Update info display
            with info_output:
                info_output.clear_output()
                print(f"Displaying {len(filtered_df)} peptides")
                if function_filter != 'All':
                    print(f"Function filter: {function_filter}")
            
            # Calculate log values
            filtered_df['log_x'] = np.log10(filtered_df[col1])
            filtered_df['log_y'] = np.log10(filtered_df[col2])
            
            # Calculate correlation
            if len(filtered_df) > 1:
                corr, _ = pearsonr(filtered_df['log_x'], filtered_df['log_y'])
            else:
                corr = float('nan')
            
            # Create scatter plot
            fig = go.Figure()
            
            # Add scatter points
            fig.add_trace(go.Scatter(
                x=filtered_df['log_x'],
                y=filtered_df['log_y'],
                mode='markers',
                name='Data Points',
                marker=dict(
                    color=color,
                    size=8,
                    line=dict(
                        color=color,
                        width=1
                    )
                ),
                hovertemplate=
                f'<b>Peptide ID:</b> %{{customdata[2]}}<br>' +
                f'<b>Function:</b> %{{customdata[3]}}<br>' +
                f'<b>{group1}:</b> %{{customdata[0]:.2e}}<br>' +
                f'<b>{group2}:</b> %{{customdata[1]:.2e}}<br>' +
                '<extra></extra>',
                customdata=np.column_stack((
                    filtered_df[col1], 
                    filtered_df[col2],
                    filtered_df['unique ID'],
                    filtered_df['function'].fillna('Unknown')
                ))
            ))
            
            # Add trendline if we have enough points
            if len(filtered_df) > 1:
                z = np.polyfit(filtered_df['log_x'], filtered_df['log_y'], 1)
                p = np.poly1d(z)
                x_range = np.linspace(filtered_df['log_x'].min(), filtered_df['log_x'].max(), 100)
                
                fig.add_trace(go.Scatter(
                    x=x_range,
                    y=p(x_range),
                    mode='lines',
                    name='Trendline',
                    line=dict(color=color, dash='dash'),
                    hovertemplate='<extra></extra>'
                ))
            
            # Update layout
            fig.update_layout(
                title=dict(
                    text=f'Correlation Plot (r = {corr:.2f})',
                    x=0.5,
                    xanchor='center'
                ),
                xaxis_title=xlabel if xlabel else group1,
                yaxis_title=ylabel if ylabel else group2,
                xaxis=dict(
                    ticktext=[f'10^{int(i)}' for i in range(int(np.floor(filtered_df['log_x'].min())), 
                                                          int(np.ceil(filtered_df['log_x'].max())) + 1)],
                    tickvals=list(range(int(np.floor(filtered_df['log_x'].min())), 
                                      int(np.ceil(filtered_df['log_x'].max())) + 1)),
                ),
                yaxis=dict(
                    ticktext=[f'10^{int(i)}' for i in range(int(np.floor(filtered_df['log_y'].min())), 
                                                          int(np.ceil(filtered_df['log_y'].max())) + 1)],
                    tickvals=list(range(int(np.floor(filtered_df['log_y'].min())), 
                                      int(np.ceil(filtered_df['log_y'].max())) + 1)),
                ),
                showlegend=False,
                width=800,
                height=800,
                template='simple_white',
                hoverlabel=dict(
                    bgcolor="white",
                    font_size=14,
                    font_family="Arial"
                )
            )
            
            # Make aspect ratio equal
            fig.update_layout(yaxis=dict(scaleanchor="x", scaleratio=1))
            
            # Show plot
            fig.show(config={
                'displayModeBar': True,
                'scrollZoom': True,
                'modeBarButtonsToRemove': ['select2d', 'lasso2d']
            })
    
    # Create widget container with vertical layout
    controls = widgets.VBox([
        widgets.VBox([
            group1_widget,
            group2_widget,
            function_widget,
            color_widget,
            xlabel_widget,
            ylabel_widget
        ]),
        info_output
    ])
    
    # Observe widget changes
    group1_widget.observe(create_plot, names='value')
    group2_widget.observe(create_plot, names='value')
    function_widget.observe(create_plot, names='value')
    color_widget.observe(create_plot, names='value')
    xlabel_widget.observe(create_plot, names='value')
    ylabel_widget.observe(create_plot, names='value')
    
    # Display widgets and create initial plot
    display(controls)
    display(plot_output)
    create_plot()

In [6]:
plot_interactive_correlation(merged_df, group_data)

Output()

In [7]:
"""
def plot_custom_correlation(df, group_data):
    # Create dropdown widgets for group selection using group names
    group_names = [group_info['grouping_variable'] for group_info in group_data.values()]
    
    group1_widget = widgets.Dropdown(
        options=group_names,
        description='Group 1:',
        value=group_names[0]
    )
    group2_widget = widgets.Dropdown(
        options=group_names,
        description='Group 2:',
        value=group_names[1]
    )
    
    # Add color picker widget
    color_widget = widgets.ColorPicker(
        concise=False,
        description='Plot Color:',
        value='#0072C6'
    )
    
    # Add text input widgets for axis labels
    xlabel_widget = widgets.Text(
        description='X Label:',
        placeholder='Enter x-axis label'
    )
    
    ylabel_widget = widgets.Text(
        description='Y Label:',
        placeholder='Enter y-axis label'
    )
        
    def plot_graph(group1, group2, color, xlabel, ylabel, df):
        # Check if same group is selected
        if group1 == group2:
            display(HTML(f'<b style="color:red;">Error: Please select different groups for comparison.</b>'))
            return
            
        col1 = f"Avg_{group1}"
        col2 = f"Avg_{group2}"
        
        if col1 not in df.columns or col2 not in df.columns:
            display(HTML(f'<b style="color:red;">Error: Skipping plot for {group1} vs {group2} as one or both columns are missing.</b>'))
            return
    
        plt.figure(figsize=(6, 6))
        
        # Drop NaN values in the relevant columns
        filtered_df = df[[col1, col2]].dropna()
        
        # Ensure only positive, non-zero values are considered
        filtered_df = filtered_df[(filtered_df[col1] > 0) & (filtered_df[col2] > 0)]
        
        # Apply log10 transformation
        log_col1 = np.log10(filtered_df[col1])
        log_col2 = np.log10(filtered_df[col2])
        
        # Calculate dynamic axis limits
        min_log = min(log_col1.min(), log_col2.min())
        max_log = max(log_col1.max(), log_col2.max())
        
        # Round to nearest integer for cleaner scale
        min_log_rounded = math.floor(min_log)
        max_log_rounded = math.ceil(max_log)
        
        # Calculate the Pearson correlation coefficient
        corr, _ = pearsonr(log_col1, log_col2)
        
        # Create scatter plot with linear regression line
        sns.regplot(
            x=log_col1, 
            y=log_col2, 
            scatter_kws={'s': 20, 'facecolors':'none', 'edgecolors':color},
            line_kws={"color": color}, 
            ci=None
        )
        
        # Set plot title and labels using custom labels if provided, otherwise use group names
        plt.xlabel(xlabel if xlabel else group1, fontsize=14)
        plt.ylabel(ylabel if ylabel else group2, fontsize=14)
        
        # Set aspect ratio to be equal
        plt.gca().set_aspect('equal', adjustable='box')
        
        # Add correlation text
        plt.text(0.05, 0.95, f'$r={corr:.2f}$', ha='left', va='top', transform=plt.gca().transAxes, fontsize=14)
        
        # Create dynamic tick marks based on rounded min and max
        log_ticks = np.arange(min_log_rounded, max_log_rounded + 1)
        
        # Set tick locations and labels
        plt.gca().set_xticks(log_ticks)
        plt.gca().set_yticks(log_ticks)
        
        # Customize the tick labels
        plt.gca().set_xticklabels([f"$10^{{{int(tick)}}}$" for tick in log_ticks], fontsize=12)
        plt.gca().set_yticklabels([f"$10^{{{int(tick)}}}$" for tick in log_ticks], fontsize=12)
        
        # Set axis limits using dynamic range
        plt.xlim(min_log_rounded, max_log_rounded)
        plt.ylim(min_log_rounded, max_log_rounded)
        
        # Remove gridlines and set minimalistic style
        sns.despine()
        
        # Show plot
        plt.tight_layout()
        plt.show()
        
    def wrapped_plot_graph(group1, group2, color, xlabel, ylabel):
        plot_graph(group1, group2, color, xlabel, ylabel, df)
        
    # Update widget interaction to include new label inputs
    widgets.interact(wrapped_plot_graph, 
                    group1=group1_widget, 
                    group2=group2_widget,
                    color=color_widget,
                    xlabel=xlabel_widget,
                    ylabel=ylabel_widget)

plot_custom_correlation(merged_df, group_data)
""";

In [8]:
def setup_correlation_exports(df, group_data):
    """
    Setup correlation analysis exports with styled download buttons
    """
    # Define CSS style
    style = """
        <style>
        .download-link {
            background-color: #4CAF50;
            border: none;
            color: white;
            padding: 10px 20px;
            text-align: center;
            text-decoration: none;
            display: inline-block;
            font-size: 14px;
            margin: 4px 2px;
            cursor: pointer;
            border-radius: 4px;
        }
        .download-link:hover {
            background-color: #45a049;
        }
        .export-section {
            margin-bottom: 20px;
            padding: 15px;
            border-radius: 5px;
            background-color: #f8f9fa;
        }
        .export-description {
            color: #666;
            margin: 5px 0 15px 0;
            font-style: italic;
        }
        </style>
    """

    def generate_download_link(content, filename, filetype='text/csv'):
        """Generate a download link for any content"""
        if isinstance(content, pd.DataFrame):
            if filetype == 'text/csv':
                content = content.to_csv(index=False)  # Remove index only for CSV
            else:
                content = content.to_csv(index=True)  # Keep index for other formats
        if isinstance(content, str):
            content = content.encode()
        b64 = base64.b64encode(content).decode()
        return f"""
            <a download="{filename}" href="data:{filetype};base64,{b64}" class="download-link" 
               title="Click to download">
                Download {filename}
            </a>
        """

    def calculate_all_correlations():
        """Calculate correlations between all group pairs"""
        correlation_results = []
        Avg_columns = {
            group_info['grouping_variable']: f"Avg_{group_info['grouping_variable']}"
            for group_info in group_data.values()
            if f"Avg_{group_info['grouping_variable']}" in df.columns
        }
        
        for (group1, col1), (group2, col2) in combinations(Avg_columns.items(), 2):
            # Filter for positive values and calculate correlation
            mask = (df[col1] > 0) & (df[col2] > 0)
            if mask.sum() > 1:
                log_col1 = np.log10(df.loc[mask, col1])
                log_col2 = np.log10(df.loc[mask, col2])
                correlation = log_col1.corr(log_col2)
                correlation = round(correlation, 3)
                correlation_results.append((group1, group2, correlation))
        
        return pd.DataFrame(correlation_results, columns=['Group 1', 'Group 2', 'Correlation'])

    def calculate_group_correlations():
        """Calculate within-group correlations and statistics"""
        correlation_dfs = {}
        
        for key, value in group_data.items():
            grouping_variable = value['grouping_variable']
            abundance_columns = value['abundance_columns']
            
            data = df[abundance_columns].copy()
            data = data[data.gt(0).all(axis=1)]
            
            if len(data) > 1:
                data = np.log10(data)
                correlation_matrix = data.corr(method='pearson')
                lower_triangle = correlation_matrix.where(
                    np.tril(np.ones(correlation_matrix.shape), k=-1).astype(bool)
                )
                
                pairs = []
                values = []
                for i in range(len(abundance_columns)):
                    for j in range(i):
                        pair_name = f"{abundance_columns[j]} vs {abundance_columns[i]}"
                        pairs.append(pair_name)
                        values.append(round(lower_triangle.iloc[i,j], 3))
                
                correlation_dfs[grouping_variable] = pd.Series(values)
        
        return correlation_dfs

    # Calculate correlations
    cross_group_correlations = calculate_all_correlations()
    within_group_correlations = calculate_group_correlations()
    
    # Create Excel for within-group correlations
    buffer = io.BytesIO()
    with pd.ExcelWriter(buffer, engine='openpyxl') as writer:
        # Combine correlations into DataFrame with numeric index
        combined_correlation_df = pd.concat(within_group_correlations, axis=1)
        
        # Calculate statistics
        min_values = combined_correlation_df.min().round(3)
        max_values = combined_correlation_df.max().round(3)
        mean_values = combined_correlation_df.mean().round(3)
        
        # Create summary DataFrame with statistics
        summary_df = pd.DataFrame({
            'Min': min_values,
            'Max': max_values,
            'Average': mean_values
        }).T
        
        # Combine correlations and summary, keeping numeric index
        combined_correlation_with_summary = pd.concat([combined_correlation_df, summary_df], axis=0)
        
        # Write summary sheet with numeric index
        combined_correlation_with_summary.to_excel(writer, sheet_name='Summary')
        
        # Write individual group sheets
        for key, value in group_data.items():
            grouping_variable = value['grouping_variable']
            abundance_columns = value['abundance_columns']
            
            if grouping_variable in within_group_correlations:
                values = within_group_correlations[grouping_variable]
                pairs = []
                for i in range(len(abundance_columns)):
                    for j in range(i):
                        pair_name = f"{abundance_columns[j]} vs {abundance_columns[i]}"
                        pairs.append(pair_name)
                
                group_df = pd.DataFrame({
                    'Pair': pairs,
                    'Correlation': values
                })
                group_df.to_excel(writer, sheet_name=grouping_variable, index=False)
        
        # Remove borders and adjust column widths
        for sheet_name in writer.sheets:
            worksheet = writer.sheets[sheet_name]
            for row in worksheet.iter_rows():
                for cell in row:
                    cell.border = None
            for column_cells in worksheet.columns:
                max_length = max(len(str(cell.value)) if cell.value is not None else 0 
                               for cell in column_cells)
                worksheet.column_dimensions[column_cells[0].column_letter].width = max_length + 2

    # Generate download sections
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Cross-group correlations CSV (without index)
    csv_filename = f"cross_group_correlations_{timestamp}.csv"
    csv_link = generate_download_link(cross_group_correlations, csv_filename)
    
    # Within-group correlations Excel (with index)
    excel_filename = f"within_group_correlations_{timestamp}.xlsx"
    excel_link = generate_download_link(
        buffer.getvalue(), 
        excel_filename, 
        'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
    )
    
    html_content = f"""
    {style}
    <div class="export-section">
        <h3>Cross-Group Correlations</h3>
        <div class="export-description">
            Download correlations between different sample groups (CSV format)
        </div>
        {csv_link}
    </div>
    
    <div class="export-section">
        <h3>Within-Group Correlations</h3>
        <div class="export-description">
            Download detailed correlation analysis for each group with summary statistics (Excel format)
        </div>
        {excel_link}
    </div>
    """
    
    display(HTML(html_content))

# Usage example:
setup_correlation_exports(merged_df, group_data)

In [9]:
def calculate_correlations_for_group_data(df, group_data):
    correlation_dfs = {}
    
    # Calculate correlations for each group
    for key, value in group_data.items():
        grouping_variable = value['grouping_variable']
        abundance_columns = value['abundance_columns']
        
        # Calculate correlation matrix
        correlation_matrix = df[abundance_columns].corr(method='pearson')
        
        # Extract lower triangle
        lower_triangle = correlation_matrix.where(np.tril(np.ones(correlation_matrix.shape), k=-1).astype(bool))
        
        # Create pairs and values for this group
        pairs = []
        values = []
        for i in range(len(abundance_columns)):
            for j in range(i):
                pair_name = f"{abundance_columns[j]} vs {abundance_columns[i]}"
                pairs.append(pair_name)
                values.append(round(lower_triangle.iloc[i,j], 3))
        
        # Create DataFrame for this group
        correlation_dfs[grouping_variable] = pd.Series(values)

    # Combine all series into a DataFrame for summary
    combined_correlation_df = pd.concat(correlation_dfs, axis=1)
    
    # Calculate statistics (rounded)
    min_values = combined_correlation_df.min().round(3)
    max_values = combined_correlation_df.max().round(3)
    mean_values = combined_correlation_df.mean().round(3)
    
    # Create summary DataFrame with statistics
    summary_df = pd.DataFrame({
        'Min': min_values,
        'Max': max_values,
        'Average': mean_values
    }).T
    
    # Combine correlations and summary
    combined_correlation_with_summary = pd.concat([combined_correlation_df, summary_df], axis=0)

    # Create Excel writer
    with pd.ExcelWriter('correlation_analysis.xlsx', engine='openpyxl') as writer:
        # Write summary sheet (original format with numeric index)
        combined_correlation_with_summary.to_excel(writer, sheet_name='Summary')
        
        # Write individual group sheets with pairs
        for key, value in group_data.items():
            grouping_variable = value['grouping_variable']
            abundance_columns = value['abundance_columns']
            
            # Calculate correlations for this group
            correlation_matrix = df[abundance_columns].corr(method='pearson')
            lower_triangle = correlation_matrix.where(np.tril(np.ones(correlation_matrix.shape), k=-1).astype(bool))
            
            # Create pairs and values
            pairs = []
            values = []
            for i in range(len(abundance_columns)):
                for j in range(i):
                    pair_name = f"{abundance_columns[j]} vs {abundance_columns[i]}"
                    pairs.append(pair_name)
                    values.append(round(lower_triangle.iloc[i,j], 3))
            
            # Create and write group-specific DataFrame
            group_df = pd.DataFrame({
                'Pair': pairs,
                'Correlation': values
            })
            group_df.to_excel(writer, sheet_name=grouping_variable, index=False)
        
        # Remove borders and adjust column widths for all sheets
        for sheet_name in writer.sheets:
            worksheet = writer.sheets[sheet_name]
            
            # Remove borders for all cells
            for row in worksheet.iter_rows():
                for cell in row:
                    cell.border = None
            
            # Auto-adjust column widths
            for column_cells in worksheet.columns:
                max_length = max(len(str(cell.value)) if cell.value is not None else 0 
                               for cell in column_cells)
                worksheet.column_dimensions[column_cells[0].column_letter].width = max_length + 2

    return combined_correlation_with_summary

# Call the function
combined_correlation_with_summary = calculate_correlations_for_group_data(merged_df, group_data)